# Flopt optique d'une vidéo

In [10]:
import numpy as np
import cv2

# Lire la premiere trame de la vidéo

In [11]:
cap = cv2.VideoCapture('VIRAT_S_050202_07_001126_001301.mp4')
ret, old_frame = cap.read()

cap.read () renvoie un booléen. Si la trame de la vidéo est lu correctement, il sera mis à vrai sinon il sera mis à faux.

# Dimensions de l'image de la vidéo

avoir la taille avec shape

In [12]:
old_frame.shape


(1080, 1920, 3)

avoir la taille avec get

In [13]:
for i in range(3,5):
    print(cap.get(i))

1920.0
1080.0


In [14]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 20.0, (old_frame.shape[1],old_frame.shape[0]))
#out = cv2.VideoWriter('output.avi',fourcc, 20.0, (1920,1080))

# Definition des paramètres de ShiTomasi et Lucas Kanade

In [15]:
# parametres ShiTomasi 
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parametres lucas kanade 
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [16]:
# Créer 100 couleurs aléatoires
color = np.random.randint(0,255,(100,3))

# Trouver les corners de la trame
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Creer un masque de dessin
mask = np.zeros_like(old_frame)

In [17]:
mask
mask.shape
mask.sum()

0

In [18]:
while(1):
    ret,frame = cap.read()
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculer le flot optique
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Selectionner les points les plus intéressants
    good_new = p1[st==1]
    good_old = p0[st==1]

    # dessiner les trajectoires
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    out.write(img)
    # le paramètre d'entrée de waitKey controlle le temps d'attente en milliseconde 
    # avant la prochaine trame de la vidéo
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # mettre à jour les images
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()
out.release()

<ipython-input-18-99d53c19d47e>:17: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
<ipython-input-18-99d53c19d47e>:18: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
